<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/stock-etc/99-02-rnn-lot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모두를 위한 머신러닝에서 가져왔습니다.
# RNN-lot


In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from io import BytesIO
import requests

# 마지막 회차번호

In [3]:
# 마지막 회차 번호
noEnd ='968' 

# url
# url = 'https://dhlottery.co.kr/gameResult.do?method=allWinExel&gubun=byWin&nowPage=&drwNoStart=1&drwNoEnd={}'.format(noEnd)

url = f'https://dhlottery.co.kr/gameResult.do?method=allWinExel&gubun=byWin&nowPage=&drwNoStart=1&drwNoEnd={noEnd}'

# user-agent
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"

'https://dhlottery.co.kr/gameResult.do?method=allWinExel&gubun=byWin&nowPage=&drwNoStart=1&drwNoEnd=968'

In [4]:
result = requests.get(url, headers={"user-agent":user_agent})

In [7]:
result

<Response [200]>

In [14]:
f = BytesIO(result.content)
# df = pd.read_excel(f,sheet_name='excel',skiprows=3)
html_result = pd.read_html(f)

In [54]:
df_raw = pd.DataFrame(html_result[1])
# df = df.drop([0])
df_raw.head()

,년도,회차,추첨일,1등당첨자수,1등당첨금액,2등당첨자수,2등당첨금액,3등당첨자수,3등당첨금액,4등당첨자수,4등당첨금액,5등당첨자수,5등당첨금액,당첨번호1,당첨번호2,당첨번호3,당첨번호4,당첨번호5,당첨번호6,당첨번호보너스
0,년도,회차,추첨일,1등,1등,2등,2등,3등,3등,4등,4등,5등,5등,당첨번호,당첨번호,당첨번호,당첨번호,당첨번호,당첨번호,당첨번호
1,년도,회차,추첨일,당첨자수,당첨금액,당첨자수,당첨금액,당첨자수,당첨금액,당첨자수,당첨금액,당첨자수,당첨금액,1,2,3,4,5,6,보너스
2,2021,968,2021.06.19,13,"1,667,729,683원",97,"37,251,694원",3116,"1,159,633원",148150,"50,000원",2329460,"5,000원",2,5,12,14,24,39,33
3,2021,967,2021.06.12,4,"5,809,776,094원",69,"56,133,103원",2475,"1,564,923원",121200,"50,000원",2042462,"5,000원",1,6,13,37,38,40,9
4,2021,966,2021.06.05,10,"2,411,303,513원",50,"80,376,784원",2352,"1,708,691원",115370,"50,000원",1980202,"5,000원",1,21,25,29,34,37,36


In [75]:
# columns = [col for i, col in enumerate(df.iloc[0])]
# df.columns = columns
# columns

# column name 설정
col_names = [fn+sn if fn != sn else fn for fn, sn in zip(df_raw.iloc[0], df_raw.iloc[1])]  
df_raw.columns = col_names

# 0, 1행 삭제
df_lot = df_raw.drop([0,1], axis=0)

# 회차 컬럼 int로 형변환
df_lot = df_lot.astype({'회차':'int32'}, copy=True)

# 회차기준 오름차준 정렬
df_lot = df_lot.sort_values('회차', ascending=True)

# 회차필드 index로 설정
df_lot.set_index('회차', inplace=True)

df_lot.head(5)

,년도,추첨일,1등당첨자수,1등당첨금액,2등당첨자수,2등당첨금액,3등당첨자수,3등당첨금액,4등당첨자수,4등당첨금액,5등당첨자수,5등당첨금액,당첨번호1,당첨번호2,당첨번호3,당첨번호4,당첨번호5,당첨번호6,당첨번호보너스
회차,,,,,,,,,,,,,,,,,,,
1,2002,2002.12.07,0,0원,1,"143,934,100원",28,"5,140,500원",2537,"113,400원",40155,"10,000원",10,23,29,33,37,40,16
2,2002,2002.12.14,1,"2,002,006,800원",2,"94,866,800원",103,"1,842,000원",3763,"100,800원",55480,"10,000원",9,13,21,25,32,42,2
3,2002,2002.12.21,1,"2,000,000,000원",0,0원,139,"1,174,100원",5940,"54,900원",73256,"10,000원",11,16,19,21,27,31,30
4,2002,2002.12.28,0,0원,1,"211,191,200원",29,"7,282,400원",2777,"152,100원",52382,"10,000원",14,27,30,31,40,42,2
5,2003,2003.01.04,0,0원,0,0원,42,"6,033,800원",3043,"166,500원",60434,"10,000원",16,24,29,40,41,42,3


In [76]:
ncount = np.zeros((1,45), dtype=float)

In [ ]:
for i in range(df.shape[0]):
  for j in range(6):
    col = int(df.iloc[i][str(j+1)])
    ncount[0][col-1] += 1.0

  if i == 0:
    X = ncount
  else:
    X = np.append(X, ncount, axis=0)

print('x.shape:', X.shape)

In [ ]:
df_y = df[['1','2','3','4','5','6']]
Y = df_y.values
print('x.shape:', X.shape)
print('y.shape:', Y.shape)
Y

In [ ]:
y_onehot = []
tmponehot = np.zeros(45).tolist()
tmponehot[3] = 1.0
y_onehot.append(tmponehot)
y_onehot.append(tmponehot)

np.array(y_onehot).shape

In [ ]:
def make_onehot(value):
  onehot = np.zeros(45).tolist()
  onehot[value-1] = 1.0
  return onehot

In [ ]:
y_onehot = []
for i in range(Y.shape[0]):
  for j in range(6):
    y_onehot.append(make_onehot(int(Y[i][j])))

y_onehot = np.array(y_onehot)
y_onehot = np.reshape(y_onehot, (Y.shape[0],6,45))
print('y_onehot.shape:', y_onehot.shape)

In [ ]:
X.shape

In [ ]:
def build_dataset(x_time_series, y_time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(y_time_series) - seq_length):
        x = x_time_series[i:i + seq_length, :]
        y = y_time_series[i + seq_length - 1, :]
        dataX.append(x)
        dataY.append(y)
    return np.array(dataX), np.array(dataY)

In [ ]:
x_data, y_data = build_dataset(X, y_onehot, 6)

In [ ]:
print('x_data.shape:', x_data.shape)
print('y_data.shape:', y_data.shape)

In [ ]:
model = keras.Sequential();
model.add(keras.layers.LSTM(units=45, input_shape=(6, 45), return_sequences=True))
model.add(keras.layers.LSTM(units=45, return_sequences=True))
model.add(keras.layers.Dense(units=45, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x_data, y_data, epochs=1500, verbose=0)

In [ ]:
x_test = x_data[-1,:]
x_test = np.delete(x_test, 0, axis=0)
x_test = np.append(x_test, X[-1:], axis=0)
x_test = x_test.reshape((1,6,45))
print('x_test.shape:', x_test.shape)

In [ ]:
pred_y = model.predict(x_test)

In [ ]:
for i in range(6):
  print(np.argmax(pred_y[0][i]) + 1)